# Feature Engineering Recipe

## Initialization

### Authentication

In [ ]:
from aoa import AoaClient, FeatureEngineeringApi
import pprint

client = AoaClient(
    aoa_url="https://vmo.local/core",
    auth_mode="device_code",
    auth_client_id="modelops",
    ssl_verify=False,
    auth_token_url=(
        "https://vmo.local/sso/realms/teradata/protocol/openid-connect/token"
    ),
    auth_device_auth_url=(
        "https://vmo.local/sso/realms/teradata/protocol/openid-connect/auth/device"
    ),
)

client.project_id = "23e1df4b-b630-47a1-ab80-7ad5385fcd8d"
fe = FeatureEngineeringApi(aoa_client=client)

### Fetch Default Dataset Connection

In [ ]:
# New API, allows users do fetch a default dataset connection
# (notice it only works for real users, service account probably don't have personal connections)

from aoa import UserAttributesApi

user_attributes_api = UserAttributesApi(aoa_client=client)
default_connection = user_attributes_api.get_default_connection()
default_connection
default_connection_id = default_connection["value"]["defaultDatasetConnectionId"]
default_connection_id

## Import Task

In [ ]:
from aoa import TrainedModelArtefactsApi
import uuid

artefacts_api = TrainedModelArtefactsApi(aoa_client=client)
artefacts = ["./requirements.txt", "./task.py"]

artefact_id = uuid.uuid4()
artefacts_api.upload_artefacts(artefact_id, artefacts)

import_request = {
    "artefactImportId": str(artefact_id),
    "name": "FE Task",
    "description": "Feature Engneering Task",
    "language": "PYTHON",
    "functionName": "run_task",
    "metadata": {
        "automation": {
            "resources": {
                "cpu": "2",
                "memory": "2g",
            },
            "engine": "PYTHON",
            "dockerImage": "artifacts.td.teradata.com/tdproduct-docker-snapshot/avmo/vmo-python-base:3.9.2",
        },
        "resourceTemplate": "Default",
    },
}

task = fe.import_task(import_request)
task_id = task["metadata"]["featureEngineeringTask"]["id"]
task_id

## Run Task

In [ ]:
# Payload to run the task
run_request = {
    "automation": {
        "dockerImage": "artifacts.td.teradata.com/tdproduct-docker-snapshot/avmo/vmo-python-base:3.9.3",
        "resources": {"memory": "2g", "cpu": "2"},
    },
    "datasetConnectionId": default_connection_id,
}

execution = fe.run(task_id, run_request)
execution

## Approve Task

In [ ]:
fe.approve(task_id, "LGTM")

## Deploy Task

In [ ]:
# Payload to deploy the task
deploy_request = {
    "engineType": "DOCKER_BATCH",
    "engineTypeConfig": {
        "dockerImage": "artifacts.td.teradata.com/tdproduct-docker-snapshot/avmo/vmo-python-base:3.9.3",
        "engine": "python-batch",
        "resources": {"memory": "500m", "cpu": "0.5"},
    },
    "language": "python",
    "datasetConnectionId": default_connection_id,
    "cron": "@manual",
}

fe.deploy(task_id, deploy_request)

### Fetch Deployment Id

In [ ]:
# fetch the deployment by task ID
from aoa import DeploymentApi

deployment = DeploymentApi(aoa_client=client)

response = deployment.find_active_by_task_id(task_id, "expandDeployment")
deployment_id = response["_embedded"]["deployments"][0]["id"]

deployment_id

## Retire Task

In [ ]:
# Payload to retire the task
retire_request = {"deploymentId": deployment_id}

fe.retire(task_id, retire_request)